# Warming Up - Keyword BERT


### 참고
- 5) 한국어 키버트(Korean KeyBERT)를 이용한 키워드 추출
    - https://wikidocs.net/159468

# 1. 환경 셋업

In [1]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 2. 함수 정의

In [2]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [3]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

# 3. 한글 예시 (1)

## 3.1. 일부 품사 (예: 명사, 형용사) 만 추출

In [4]:
doc = """
드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
"향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
"""

doc = '주꾸미 오징어 문어 요런거 느므느므 좋아하는데 혼자 사는 싱글족이라 주꾸미 땡겨도 막상 근처에 갈데가 없고,\
       볶음류가 은근 2인분 이상 주문해야되는곳도 많아서ㅠㅠ 요즘 한끼 먹기 좋게 포장된 주꾸미를 열심히 섭렵 중에 있답니닷! ^^ \
       한끼용이지만 마치 두끼인것처럼 든든하게 먹기 위해서 좋아하는거 맘껏 다 준비해서 만들었어요ㅎㅎㅎ\
       1. 중국식 넓적 당면 불려 두고 2. 함께 주문한 왕통통 새우도 넣고 \
       3. 볶음류에 깻잎 또 필수 중에 필수죠ㅋㅋ 4. 거기다가 매운고추 더 뿌려서 \
       5. 고소함 띠드 한장 싸악 올려서 먹음 채고!!! 채고!!! 오늘 받자마자 저녁으로 만들어 먹어보니 \
       홍대주꾸미가 다른 주꾸미 보다 좋았던게 익는 순간 쪼그라 들면서 반토막나는데가 진짜 너무 많았거든요? ㅠㅠ \
       실망 실망 대실망ㅠㅠ 근데 여긴 사이즈가 변함없이 통통하니 쫄깃쫄깃 씹는 재미,식감을 즐기기도 너무 좋더라구요. >.< \
       딱 좋아요! 맘에 들었어요! 쟁여두고 사먹을 아이템으로 등극했어요~*^^*'

In [5]:
word = 'Noun'

word in "Hello"

False

In [6]:
from konlpy.tag import Okt
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_POS = ' '.join([word[0] for word in tokenized_doc if word[1] in "Noun Adjective Number Verb"])
# tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'] or word[1]=='Adjective')
# tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('Taggings Sample: \n',tokenized_doc[:50])
print('POS Samples  :\n',tokenized_POS)

Taggings Sample: 
 [('주꾸미', 'Noun'), ('오징어', 'Noun'), ('문어', 'Noun'), ('요런', 'Modifier'), ('거', 'Noun'), ('느므느므', 'Noun'), ('좋아하는데', 'Adjective'), ('혼자', 'Noun'), ('사는', 'Verb'), ('싱글', 'Noun'), ('족', 'Noun'), ('이라', 'Josa'), ('주꾸미', 'Noun'), ('땡겨도', 'Verb'), ('막상', 'Noun'), ('근처', 'Noun'), ('에', 'Josa'), ('갈데가', 'Verb'), ('없고', 'Adjective'), (',', 'Punctuation'), ('볶음', 'Noun'), ('류', 'Noun'), ('가', 'Josa'), ('은근', 'Noun'), ('2', 'Number'), ('인분', 'Noun'), ('이상', 'Noun'), ('주문', 'Noun'), ('해야', 'Verb'), ('되는', 'Verb'), ('곳도', 'Noun'), ('많아서', 'Adjective'), ('ㅠㅠ', 'KoreanParticle'), ('요즘', 'Noun'), ('한', 'Determiner'), ('끼', 'Noun'), ('먹기', 'Noun'), ('좋게', 'Adjective'), ('포장', 'Noun'), ('된', 'Verb'), ('주꾸미', 'Noun'), ('를', 'Josa'), ('열심히', 'Adverb'), ('섭렵', 'Noun'), ('중', 'Noun'), ('에', 'Josa'), ('있답니', 'Adjective'), ('닷', 'Noun'), ('!', 'Punctuation'), ('^^', 'Punctuation')]
POS Samples  :
 주꾸미 오징어 문어 거 느므느므 좋아하는데 혼자 사는 싱글 족 주꾸미 땡겨도 막상 근처 갈데가 없고 볶음 류 은근 2 인분 이상 주문 해야 되는 곳도 많아서 요즘 끼 먹기

## 3.2. Bi-gram (두개 연속 단어), Tri-gram (세개 연속 단어) 를 구함

In [7]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_POS])
candidates = count.get_feature_names_out()

print('# of bigram & trigram :',len(candidates))
print('\nbigram and trigram Samples :\n',candidates[:100])

# of bigram & trigram : 204

bigram and trigram Samples :
 ['갈데가 없고' '갈데가 없고 볶음' '거기 다가' '거기 다가 매운' '고소함 띠드' '고소함 띠드 한장' '고추 뿌려서'
 '고추 뿌려서 고소함' '곳도 많아서' '곳도 많아서 요즘' '근처 갈데가' '근처 갈데가 없고' '기도 좋더라구요'
 '기도 좋더라구요 좋아요' '깻잎 필수' '깻잎 필수 필수' '넣고 볶음' '넣고 볶음 깻잎' '느므느므 좋아하는데'
 '느므느므 좋아하는데 혼자' '는데가 진짜' '는데가 진짜 많았거든요' '다가 매운' '다가 매운 고추' '다른 주꾸미'
 '다른 주꾸미 보다' '당면 불려' '당면 불려 두고' '되는 곳도' '되는 곳도 많아서' '두고 주문' '두고 주문 통통'
 '든든하게 먹기' '든든하게 먹기 위해' '들면서 반토막' '들면서 반토막 는데가' '들었어요 쟁여두' '들었어요 쟁여두 사먹을'
 '땡겨도 막상' '땡겨도 막상 근처' '띠드 한장' '띠드 한장 올려서' '마치 든든하게' '마치 든든하게 먹기' '막상 근처'
 '막상 근처 갈데가' '만들어 먹어' '만들어 먹어 보니' '만들었어요 중국' '만들었어요 중국 당면' '많아서 요즘'
 '많아서 요즘 먹기' '많았거든요 실망' '많았거든요 실망 실망' '맘껏 준비' '맘껏 준비 해서' '매운 고추'
 '매운 고추 뿌려서' '먹기 위해' '먹기 위해 좋아하는거' '먹기 좋게' '먹기 좋게 포장' '먹어 보니' '먹어 보니 홍대'
 '먹음 채고' '먹음 채고 채고' '문어 느므느므' '문어 느므느므 좋아하는데' '반토막 는데가' '반토막 는데가 진짜'
 '받자마자 저녁' '받자마자 저녁 만들어' '변함 통통하니' '변함 통통하니 쫄깃쫄깃' '보니 홍대' '보니 홍대 주꾸미'
 '보다 좋았던게' '보다 좋았던게 익는' '볶음 깻잎' '볶음 깻잎 필수' '볶음 은근' '볶음 은근 인분' '불려 두고'
 '불려 두고 주문' '뿌려서 고소함' '뿌려서 고소함 띠드'

## 3.3. Pre-Trained 로딩 로딩

In [8]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

## 3.4. 자연어를 모델에 입력하여 모델 Output embedding 얻기

In [9]:

doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [10]:
print("doc_embedding shape: ", doc_embedding.shape)
print("candidate_embeddings shape: ", candidate_embeddings.shape)

doc_embedding shape:  (1, 768)
candidate_embeddings shape:  (204, 768)


# 3.5. 원문 (Doc) 과 Bi-gram 및 Tri-gram 의 유사한 것 찾기

### 3.5.1 Cosine Similarity

In [11]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['볶음 깻잎', '넣고 볶음 깻잎', '새우도 넣고 볶음', '볶음 은근', '볶음 은근 인분']


### 3.5.2 Max Sum Similarity
- "후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다."
    - https://wikidocs.net/159468

In [12]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['고추 뿌려서 고소함', '매운 고추', '많아서 요즘 먹기', '새우도 넣고 볶음', '볶음 은근']

### 3.5.3 Maximal Marginal Relevance
- "MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드/키프레이즈를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다. 먼저 문서와 가장 유사한 키워드/키프레이즈를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택합니다."
    - https://wikidocs.net/159468

In [13]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['볶음 은근 인분', '많아서 요즘 먹기', '새우도 넣고 볶음', '중국 당면 불려', '넣고 볶음 깻잎']

In [14]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['볶음 은근 인분', '실망 실망', '중국 당면', '좋아하는데 혼자 사는', '다른 주꾸미']